<a href="https://colab.research.google.com/github/NagaoTadashi/Ledge-recommend/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim==4.0.1

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [101]:
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

File ‘../data/ml-10m.zip’ already there; not retrieving.

Archive:  ../data/ml-10m.zip


映画情報

In [102]:
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

In [103]:
# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))

タグ情報

In [104]:
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

In [105]:
# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()

In [106]:
# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

In [107]:
# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

学習データの作成

In [108]:
# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movies["tag_genre"] = movies["tag"].fillna("").apply(list) + movies["genre"].apply(list)
movies["tag_genre"] = movies["tag_genre"].apply(lambda x: list(map(str, x)))

In [111]:
movies.head()

,movie_id,title,genre,tag,tag_genre
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat...","[pixar, pixar, pixar, animation, pixar, animat..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[for children, game, animals, joe johnston, ro...","[for children, game, animals, joe johnston, ro..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[funniest movies, comedinha de velhinhos engra...","[funniest movies, comedinha de velhinhos engra..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[girl movie],"[girl movie, Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy],"[steve martin, pregnancy, remake, steve martin...","[steve martin, pregnancy, remake, steve martin..."


In [112]:
# タグとジャンルデータを使って、ldaを学習する
tag_genre_data = movies.tag_genre.tolist()